In [60]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF

from arcgis.gis import GIS, Item

import sys
sys.path.append('..')

import geometry_monkeypatch
import reach_tools

In [164]:
import importlib
importlib.reload(reach_tools)
importlib.reload(geometry_monkeypatch)

<module 'geometry_monkeypatch' from '../geometry_monkeypatch.py'>

In [160]:
waters = reach_tools.WATERS()
gis = GIS()

In [165]:
def update_trace_with_nhdplus(self):

    # get the snapped and corrected reach locations for the put-in
    self.putin.snap_to_nhdplus()

    # use the EPA navigate service to trace downstream
    trace_polyline = waters.get_downstream_trace_polyline(self.putin.nhdplus_reach_id, self.putin.nhdplus_measure)

    # project the takeout geometry to the same spatial reference as the trace polyline
    takeout_geom = self.takeout.geometry.match_spatial_reference(self.takeout.geometry)

    # snap the takeout geometry to the hydroline
    takeout_geom = takeout_geom.snap_to_line(trace_polyline)

    # update the takeout in the reach
    self.takeout.set_geometry(takeout_geom)

    # trim off the tail below the takeout
    self['SHAPE'] = trace_polyline.split_at_point(takeout_geom)[0]
    
    return True

reach_tools.Reach.update_trace_with_nhdplus = update_trace_with_nhdplus

In [166]:
ldub = reach_tools.Reach.get_from_aw(2156)
ldub.update_trace_with_nhdplus()

True

In [167]:
ldub.as_feature

{"geometry": {"paths": [[[-121.633094507621, 45.7953236766543], [-121.632808521595, 45.7949415287295], [-121.632717521896, 45.7943735286163], [-121.6334955218, 45.7932515290357], [-121.633564521385, 45.7930455285292], [-121.633434521685, 45.7927295292453], [-121.632061521322, 45.7921535287633], [-121.631961388107, 45.7919015288334], [-121.632457388497, 45.7912375287898], [-121.63229038799, 45.7909205285604], [-121.631244388317, 45.7902535292779], [-121.631282388271, 45.7898645293255], [-121.632885388449, 45.7883655285472], [-121.633014321553, 45.7874455292887], [-121.632686321716, 45.7869235285989], [-121.631969321926, 45.7865345286465], [-121.629261188053, 45.7854585959384], [-121.628185187896, 45.7846535956878], [-121.627140188269, 45.7835315952079], [-121.626941121535, 45.7830555957401], [-121.627071121235, 45.7822085953513], [-121.628055121646, 45.7815485954922], [-121.628444121598, 45.7814335955849], [-121.628711121321, 45.7801525958665], [-121.628680988636, 45.7792865955009], [-1